# Semantic Evaluation - Experiment 05
The objective of this trial is to expand the SCA_index (i.e., Semantic Content Analysis Index) to a full word embedding, setting a subjective or objective load for each word.

## Introduction

### Libraries

In [1]:
## Data analysis packages:
import pandas as pd
import numpy as np
from math import isnan  #Verifies if a given value is numerical.

In [2]:
## Visualization packages:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

### Definitions

In [3]:
## Forcing Pandas to display any number of elements
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_seq_items', None)
pd.set_option('display.width', 2000)
pd.set_option('display.max_colwidth', None)

In [4]:
## Based on: https://stackoverflow.com/questions/25351968/how-can-i-display-full-non-truncated-dataframe-information-in-html-when-conver
def print_full(x):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.2f}'.format)
    pd.set_option('display.max_colwidth', None)
    print(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')

## Exploring the SpaCy Word Embeddings: 
Also using Spacy library: https://spacy.io/
> !pip install -U spacy  
> !python -m spacy download en_core_web_sm  
> !python -m spacy download en_core_web_lg

Some instructions on how to use it:  
https://spacy.io/usage/spacy-101

In [5]:
## Importing SpaCy library:
import spacy

# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_lg")

In [6]:
## We check below that this model has 514.157 keys and vectors, respectively.
nlp.meta['vectors']

{'width': 300,
 'vectors': 514157,
 'keys': 514157,
 'name': 'en_vectors',
 'mode': 'default'}

In [7]:
## Again, checking the number of keys.
nlp.vocab.vectors.n_keys

514157

In [9]:
## Getting the word embedding: data (i.e., the matrix containing the vector values for each word)
word_embedding = nlp.vocab.vectors.data

## Verifying the shape of the word embedding matrix:
word_embedding.shape

(514157, 300)

--- 
### Finding the words associated with the embedding:

In [10]:
## Extracting the words associated with each index:
index = nlp.vocab.vectors.keys()
words_associated = [nlp.vocab[i].text for i in index]

In [11]:
## Checking the word in position 514156, wich is "Lahouaiej":
words_associated[514156]

'Lahouaiej'

In [12]:
## Finding the respective row (index) for a given word:
rows = nlp.vocab.vectors.find(keys=["cat", "dog", "Lahouaiej"])
rows

array([  3201,   1147, 514156], dtype=int32)

---
## SCA - Glasgow Norms
* Read the SCA from Glasgow Norms;  
* Import F_s and F_o from the previous study;  
* Train the MLP classifier.

In [15]:
df_factors = pd.read_csv('../data/df_factors.csv', sep=';')
df_factors.head()

,words,F_Objectivity,F_Subjectivity,F_Context
0,abattoir,0.512527,0.380603,0.960466
1,abbey,0.714765,0.240456,0.696198
2,abbreviate,0.286952,0.171052,0.767043
3,abdicate,0.144736,0.384300,0.863127
4,abdication,0.167654,0.334086,0.896733


In [22]:
SCA_words = [word for word in df_factors.words]

In [24]:
SCA_embedding_rows =  nlp.vocab.vectors.find(keys=SCA_words)

In [29]:
len(SCA_embedding_rows)

5553

> Separating the SCA-GlasgowNorms data into train and test:

In [94]:
from sklearn.model_selection import train_test_split

# Separar os dados em conjuntos de treino (70%) e teste (30%)
train_df, test_df = train_test_split(df_factors, test_size=0.3, random_state=42)

In [95]:
# Função para criar os conjuntos de treino e resposta
def create_data(dataframe):
    X = {}
    Y = {}
    
    for index, row in dataframe.iterrows():
        word = row['words']
        f_objectivity = row['F_Objectivity']
        f_subjectivity = row['F_Subjectivity']

        if word in nlp.vocab:
            indice = nlp.vocab.strings[word]
            vetor_embedding = word_embedding[index]
            X[word] = vetor_embedding
            Y[word] = {'F_Objectivity': f_objectivity, 'F_Subjectivity': f_subjectivity}

    return pd.DataFrame.from_dict(X, orient='index'), pd.DataFrame.from_dict(Y, orient='index')

In [100]:
# Creating train and test datasets:
X_train, Y_train = create_data(train_df)
X_test, Y_test = create_data(test_df)

# Exibir as dimensões dos conjuntos de treino e teste
print("Train data dimension:")
print("X_train:", X_train.shape)
print("Y_train:", Y_train.shape)

print("\nTest data dimension:")
print("X_test:", X_test.shape)
print("Y_test:", Y_test.shape)

Train data dimension:
X_train: (3288, 300)
Y_train: (3288, 2)

Test data dimension:
X_test: (1392, 300)
Y_test: (1392, 2)


In [101]:
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
airplane,-3.900300,3.93430,0.65395,1.29990,2.83170,1.508200,6.75800,5.364900,-2.36660,-3.08060,7.3809,0.22831,-2.47630,0.017361,-0.68943,-2.390400,-0.060711,-3.533500,1.810800,-2.0253,-1.97780,-1.41780,-1.9544,0.10393,-2.83050,-3.57070,-2.80240,-0.88663,-3.289400,1.66370,5.91670,-0.091764,-2.84720,-5.14470,-2.39620,1.18960,-3.490000,1.823300,4.96860,2.07140,-2.244400,5.36050,-0.95761,4.29600,-0.21680,0.62048,1.73920,-5.99340,-2.89730,3.93680,-2.401700,1.49590,-0.054549,-6.26560,-4.5234,0.53169,1.63590,0.89828,1.03740,2.017600,-2.9216,1.64230,-3.3777,-0.49631,3.28690,2.7791,-2.59510,-3.8540,1.46620,2.8515,4.42560,3.98090,-0.73355,-0.119020,0.89358,0.70556,-2.02380,2.77660,0.26854,4.13280,-5.96310,-1.10990,1.34010,-1.51770,0.316740,-3.018500,-5.17970,-3.0217,-0.04403,0.049687,-1.47610,4.24500,3.33700,-1.33770,5.05770,1.07830,-0.016471,-2.56510,-1.393600,0.63940,1.57260,-0.011109,-1.69850,4.3501,-5.49180,5.04830,-2.20180,-0.41809,-0.78404,-0.66416,1.15280,1.23260,-1.30860,0.31578,-1.19400,3.92370,0.46197,1.06130,-0.319790,-2.38780,2.12210,-1.80540,-1.42700,0.53254,-3.0410,-2.34490,-1.62040,-2.96710,4.63050,0.16084,-4.8255,-0.29363,3.13940,-1.95800,-1.29290,4.14060,-0.038602,1.524500,5.9246,-2.75440,-2.19690,-0.31086,4.74790,-1.76610,-0.22267,-0.300000,-1.75270,-0.050056,-2.55650,1.28360,0.41696,2.88090,0.355010,-0.87239,-1.04280,0.307970,4.1562,-1.008100,-1.14420,-1.09840,-1.91330,-1.54030,-3.62340,0.42826,-5.00650,-0.49866,-4.22030,2.79070,-0.38192,3.9372,-1.97130,3.24010,-0.78560,-0.079338,2.72100,-2.73960,-4.29080,-1.70190,-2.51120,0.14885,-2.91820,-0.40600,6.545700,-3.012900,-0.27173,-0.104410,-5.43240,-1.326000,3.23230,3.2203,1.94580,1.15240,-1.384000,2.16370,-2.54230,2.771200,-0.78878,-3.597100,-0.001075,2.14810,-2.43480,-8.08430,-3.99480,-3.24140,0.19680,1.93320,-1.234800,3.02840,-3.30660,0.92006,0.59873,0.294280,-1.83340,1.32660,1.281800,4.80140,0.95566,-2.15410,-1.82020,-0.559350,-2.19780,-1.86960,-0.83113,-0.29340,-3.46630,-3.49460,-1.11120,1.393900,2.5032,1.39520,0.29229,-2.8486,0.038245,2.59330,0.27597,2.113800,1.97000,5.78300,-7.13120,-1.47090,-1.117400,-0.45468,-1.92740,-1.14920,1.184000,-3.34210,-2.28830,1.93360,3.3205,5.47850,1.66630,-2.19340,-7.69760,1.85860,-0.22897,-2.68230,1.09740,0.35450,0.83341,0.53664,-0.57928,5.3247,1.9213,1.56950,-0.74805,-1.08930,3.24310,4.85570,-7.01540,1.43420,5.40910,-4.71290,-0.548970,-0.41088,0.66925,-2.922000,0.54973,-0.66148,-1.50200,1.3905,2.269300,1.57630,-1.49930,1.29760,3.150300,-3.51840,0.673510,3.10910,-6.3776,2.24180,-0.36937,1.10960,4.207400,1.66340,0.52245,3.5314,0.79304,0.11406,-5.10790,1.122000
coarse,2.042700,-2.06910,-1.59100,0.42295,3.51720,-0.042939,1.57450,5.945900,-7.09320,-3.81480,11.7570,4.35480,-3.72050,4.313600,0.26884,-0.443220,4.132300,1.703300,-3.872900,1.4727,-1.43550,0.94668,-3.8571,1.27040,-4.54170,-2.14030,1.29390,-1.74510,-5.526400,3.00230,2.44820,2.639000,-1.52310,2.31340,1.35940,-

In [102]:
Y_train

,F_Objectivity,F_Subjectivity
airplane,0.960395,0.463949
coarse,0.401438,0.265841
persecution,0.319377,0.678163
moment,0.139091,0.413798
responsible,0.289618,0.654414
...,...,...
prawn,0.952891,0.110865
tweezers,0.953419,0.193659
university,0.835052,0.695757
wasteful,0.200742,0.492842


#### Binarizing Y_train and Y_test
Once we run the first MLP model, the performance wasn't over 54%. 
In this Section, we will binarize the semantic factor values following the median values.

In [99]:
# Binarizing through list comprehension
Y_train['F_Objectivity'] = ['high' if f_objectivity >= 0.565 else 'low' for f_objectivity in Y_train['F_Objectivity']]
Y_train['F_Subjectivity'] = ['high' if f_subjectivity >= 0.392 else 'low' for f_subjectivity in Y_train['F_Subjectivity']]

Y_test['F_Objectivity'] = ['high' if f_objectivity >= 0.565 else 'low' for f_objectivity in Y_test['F_Objectivity']]
Y_test['F_Subjectivity'] = ['high' if f_subjectivity >= 0.392 else 'low' for f_subjectivity in Y_test['F_Subjectivity']]

In [103]:
# Binarizing through list comprehension
Y_train['F_Objectivity'] = [1 if f_objectivity >= 0.565 else 0 for f_objectivity in Y_train['F_Objectivity']]
Y_train['F_Subjectivity'] = [1 if f_subjectivity >= 0.392 else 0 for f_subjectivity in Y_train['F_Subjectivity']]

Y_test['F_Objectivity'] = [1 if f_objectivity >= 0.565 else 0 for f_objectivity in Y_test['F_Objectivity']]
Y_test['F_Subjectivity'] = [1 if f_subjectivity >= 0.392 else 0 for f_subjectivity in Y_test['F_Subjectivity']]

In [104]:
Y_train

,F_Objectivity,F_Subjectivity
airplane,1,1
coarse,0,0
persecution,0,1
moment,0,1
responsible,0,1
...,...,...
prawn,1,0
tweezers,1,0
university,1,1
wasteful,0,1


---
### Training a MLP Classifier for word semantic content

In [75]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer

In [113]:
# Define a new MLP architecture
model = Sequential([
    Dense(256, activation='relu', input_shape=(300,)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(2, activation='softmax')  # 2 neurons for binary classification with softmax activation
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Use categorical_crossentropy for categorical data
              metrics=['accuracy'])

In [114]:
# Print model summary
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_39 (Dense)            (None, 256)               77056     
                                                                 
 dense_40 (Dense)            (None, 128)               32896     
                                                                 
 dense_41 (Dense)            (None, 64)                8256      
                                                                 
 dense_42 (Dense)            (None, 32)                2080      
                                                                 
 dense_43 (Dense)            (None, 2)                 66        
                                                                 
Total params: 120354 (470.13 KB)
Trainable params: 120354 (470.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


> Como converti a saída em dados categóricos, é preciso antes utilizar OneHotEncoder:

In [124]:
## Como já foram transformados anteriormente em [0,1], preciso apenas torná-los como lista:
Y_train_array = Y_train.to_numpy()
Y_test_array = Y_test.to_numpy()

X_train_array = X_train.to_numpy()
X_test_array = X_test.to_numpy()

# Print the first few elements to verify
print(Y_train_array[:5])  # Print the first 5 elements


[[1 1]
 [0 0]
 [0 1]
 [0 1]
 [0 1]]


In [79]:
# Convert multilabel categorical labels to binary vectors
multi_label_binarizer = MultiLabelBinarizer()
Y_train_encoded = multi_label_binarizer.fit_transform(Y_train)
Y_test_encoded = multi_label_binarizer.transform(Y_test)

In [85]:
len(Y_train_encoded)

2

In [86]:
Y_train_encoded

array([[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1],
       [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [108]:
Y_train

,F_Objectivity,F_Subjectivity
airplane,1,1
coarse,0,0
persecution,0,1
moment,0,1
responsible,0,1
...,...,...
prawn,1,0
tweezers,1,0
university,1,1
wasteful,0,1


In [125]:
X_train_array

array([[ -3.9003  ,   3.9343  ,   0.65395 , ...,   0.11406 ,  -5.1079  ,
          1.122   ],
       [  2.0427  ,  -2.0691  ,  -1.591   , ...,  -3.8809  ,  -5.2335  ,
          3.8502  ],
       [  0.068316,   4.3433  ,  -5.5537  , ...,   2.2451  ,  -1.9138  ,
         -0.074928],
       ...,
       [ -1.0686  , -10.114   ,   4.0364  , ...,   6.1114  ,  -5.9124  ,
         10.933   ],
       [ -5.8985  ,  -2.8247  ,   0.85307 , ...,  -0.22011 ,   3.1951  ,
          0.082503],
       [  2.5277  ,   2.4585  ,   1.4097  , ...,  -3.3495  ,  -4.2642  ,
          0.89491 ]], dtype=float32)

In [128]:
# Train the model
history = model.fit(X_train_array, Y_train_array, epochs=50, batch_size=16, validation_split=0.2)

Epoch 1/50
165/165 [==============================] - 0s 2ms/step - loss: 0.1686 - accuracy: 0.7859 - val_loss: 2.5079 - val_accuracy: 0.4818
Epoch 2/50
165/165 [==============================] - 0s 2ms/step - loss: 0.1089 - accuracy: 0.7875 - val_loss: 2.3956 - val_accuracy: 0.5000
Epoch 3/50
165/165 [==============================] - 0s 2ms/step - loss: 0.0577 - accuracy: 0.7806 - val_loss: 2.9911 - val_accuracy: 0.5410
Epoch 4/50
165/165 [==============================] - 0s 2ms/step - loss: 0.0505 - accuracy: 0.7844 - val_loss: 2.7022 - val_accuracy: 0.5486
Epoch 5/50
165/165 [==============================] - 0s 2ms/step - loss: 0.0321 - accuracy: 0.7890 - val_loss: 3.3800 - val_accuracy: 0.4924
Epoch 6/50
165/165 [==============================] - 0s 2ms/step - loss: 0.0284 - accuracy: 0.7684 - val_loss: 3.4306 - val_accuracy: 0.5228
Epoch 7/50
165/165 [==============================] - 0s 2ms/step - loss: 0.0330 - accuracy: 0.7897 - val_loss: 3.4110 - val_accuracy: 0.5334
Epoch 

In [129]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_array, Y_test_array)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

44/44 [==============================] - 0s 1ms/step - loss: 3.9207 - accuracy: 0.4504
Test Accuracy: 45.04%


In [65]:
# Treina o modelo
history = model.fit(X_train, Y_train,
                    epochs=50,
                    batch_size=32,
                    validation_split=0.1)  # Usamos parte dos dados de treino como validação

# Avalia o modelo com os dados de teste
loss, accuracy = model.evaluate(X_test.values, Y_test.values)
print(f"Acurácia do modelo nos dados de teste: {accuracy * 100:.2f}%")

Epoch 1/50


UnimplementedError: Graph execution error:

Detected at node categorical_crossentropy/Cast defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "c:\Users\tiago\OneDrive - UNIVALI\PhD\atividades de pesquisa\semantic_similarity\.venv\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "c:\Users\tiago\OneDrive - UNIVALI\PhD\atividades de pesquisa\semantic_similarity\.venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "c:\Users\tiago\OneDrive - UNIVALI\PhD\atividades de pesquisa\semantic_similarity\.venv\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "c:\Users\tiago\OneDrive - UNIVALI\PhD\atividades de pesquisa\semantic_similarity\.venv\Lib\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2032.0_x64__qbz5n2kfra8p0\Lib

### Utilizando XGBoosting for multilabel:

### Utilizando PyCaret:

In [131]:
from pycaret.classification import *

ImportError: cannot import name '_PredictScorer' from 'sklearn.metrics._scorer' (c:\Users\tiago\OneDrive - UNIVALI\PhD\atividades de pesquisa\semantic_similarity\.venv\Lib\site-packages\sklearn\metrics\_scorer.py)